In [1]:
import backend
import json
import generate_page as page
initial_prompt = """
Generate YC’s Co-founder Matching website. It allows entrepreneurs to create a profile, and find the contact information of other entrepreneurs.
It should feature the following pages:
1. A home landing page advertising the Co-founder matching site, which allows the user to start creating an account using the built-in clerk authentication buttons, and shows marketing items advertising how many prestigious Stanford/MIT cofounders are on here. 
2. Next, after the user creates an account by entering their email, they are taken to a page where they create their own profile. They fill out their profile information, which includes the following: their name, a one liner on their background, a link to a linkedin profile (which is hidden from others until matched), and a checkbox on if they are a technical cofounder or not.
3. After their profile is created, they then can view other profiles and interact with them on the match page. Here, they scroll through a list of other user profiles who do not have a match yet. If they are interested, they can press a match button that sends them a match request as well as a short message.
4. Users can also view their match requests, and either approve or reject them. If a request is rejected, it is removed from the matches. If they approve, the request gets moved to a match section, where they can add the other person on linkedin using their linkedin link.
5. Users can also edit their profile as well. If they found a cofounder, they can mark their profile as matched, and they will be removed from the search.

Schema:
User: email, name, linkedin, description, technical / non technical, available / not available
Match: User from, User to, Accepted / rejected, request message, accept message

"""
schema_path = f'generated/schema.json'
schema_page_path= f'test/convex/schema.ts'
actions_path = 'generated/actions.json'
crud_path = 'generated/crud.ts'
backend_json = 'generated/backend.json'
backend_path = 'test/convex/backend.ts'
page_path = 'generated/page_schema.json'
test_path = 'test/app'
faker_path = 'test/convex/faker.ts'


In [2]:
backend.write_to_file(backend.create_schema_structure(initial_prompt), schema_path)

Here is the schema for the YC Co-founder Matching website:

<jsonSchema>
{
  "description": "The YC Co-founder Matching website allows entrepreneurs to create a profile and find contact information of other entrepreneurs to potentially match with as co-founders. Users can create a profile, view and match with other profiles, manage match requests, and edit their profile.",
  "features_description": "1. Home landing page advertising the site with Clerk authentication to create account\n2. Profile creation page to fill out name, background, LinkedIn profile, and technical cofounder status\n3. Match page to view other available user profiles and send match requests with message\n4. View and manage incoming match requests, approve to connect on LinkedIn or reject\n5. Edit profile page and mark profile as matched once co-founder is found",
  "schema_tables": [
    {
      "name": "users",
      "docs": "Stores user profile information",
      "fields": [
        {
          "name": "email",

In [3]:
schema = backend.read(schema_path)
backend.write_to_file(backend.create_actions(f"Create it for the following structure: \n<applicationStructure>\n{schema}\n</applicationStructure>"), actions_path)

Here is the generated JSON schema for the backend actions based on the provided application structure:

<jsonSchema>
{
  "actions_description": "The backend actions will handle creating and updating user profiles, retrieving user profiles for the match page, creating and managing match requests between users.",
  "query_actions_required": "Query actions are needed to retrieve a user's own profile, get a list of available user profiles for the match page, and get a user's incoming and outgoing match requests.",
  "query_actions": [
    {
      "name": "getUserProfile",
      "where_used": "Edit Profile page",
      "docs": "Retrieves the profile of the currently logged in user.",
      "requires_auth": true,
      "arguments": [],
      "returns": "The user's profile data",
      "return_type": "User object",
      "workflow_steps": [
        {
          "step": "Get the currently logged in user's ID from Clerk auth"
        },
        {
          "step": "Query the 'users' table for th

In [3]:
# entirely compiler
schema = json.loads(backend.read(schema_path))
# actions = backend.read(actions_path)
# schema_page = backend.create_schema(schema, schema_page_path)
# crud_page = backend.create_crud(schema, crud_path)
# actions_code = backend.create_actions_code(actions, schema_page, crud_page, backend_json)
# actions_page = backend.create_actions_page(json.loads(actions), json.loads(actions_code),crud_page, backend_path)

In [4]:
faker_data = backend.create_faker_data_code(schema, faker_path)

Here's the fake data generator based on the provided schema:

```ts
import { faker } from '@faker-js/faker';
import { internalMutation } from "./_generated/server";

export const createFake = internalMutation(async (ctx) => {
  faker.seed();

  for (let i = 0; i < 20; i++) {
    const userId = await ctx.db.insert("users", {
      email: faker.internet.email(),
      name: faker.person.fullName(),
      linkedin: faker.internet.url(),
      description: faker.lorem.paragraph(),
      isTechnical: faker.datatype.boolean(),
      isAvailable: faker.datatype.boolean(),
      tokenIdentifier: faker.person.zodiacSign()
    });

    const numMatches = faker.number.int({ min: 0, max: 5 });
    // get all users
    const users = (await ctx.db.query("users").filter((q) => q.neq(q.field("_id"), userId)).collect())
    for (let j = 0; j < numMatches; j++) {
      // select random user to match with
      const toUserId = users[Math.floor(Math.random() * users.length)]._id
      
      // create ma

In [ ]:
# creating the page structure
schema = backend.read(schema_path)
actions = backend.read(actions_path)
backend.write_to_file(backend.create_page_structure("Task: " + initial_prompt + "\n\nHere is the database schema for the avaiable database objects you are to use: \n<Schemas>\n" + str(schema) + "\n</Schemas>\n\n Here are the backend queries and mutations that you can call: \n<backend>" + str(actions) + "\n</backend>\n\nNow, generate the page layout strictly in the json format described above." ), page_path)

In [7]:
# alternative way to make page structure: simplified version
page.write(page_path, page.generate_component_list(schema_path, actions_path))

{
    "pages": [
        {
            "url": "/",
            "title": "Home",
            "description": "Landing page advertising the site with Clerk authentication to create account"
        },
        {
            "url": "/createProfile",
            "title": "CreateProfile",
            "description": "Profile creation page to fill out name, background, LinkedIn profile, and technical cofounder status"  
        },
        {
            "url": "/match",
            "title": "Match",
            "description": "Match page to view other available user profiles and send match requests with message"
        },
        {
            "url": "/matchRequests",
            "title": "MatchRequests", 
            "description": "View and manage incoming match requests, approve to connect on LinkedIn or reject"
        },
        {
            "url": "/editProfile",
            "title": "EditProfile",
            "description": "Edit profile page and mark profile as matched once co-founder 

In [2]:
# generate all the parts
pages = json.loads(backend.read(page_path))["pages"]
# page.generate_pages(test_path, pages, schema_path, actions_path)
# page.generate_page(test_path, pages, pages[4], schema_path, actions_path, 0)

In [3]:
project = page.Project(test_path, pages, schema_path, actions_path)
project.print_pages_menu()

0 Home (g)
1 CreateProfile (g)
2 Match (g)
3 MatchRequests (g)
4 EditProfile (g)


In [4]:
project.print_page_menu(3)

MatchRequests (g)
(0) Header (g) (s)
(1) IncomingRequestList (g) (ns)
(2) OutgoingRequestList (g) (ns)
(3) MatchRequestCard (g) (ns)
(4) Footer (g) (s)


In [26]:
# project.edit_page(1, "")
project.edit_component(3, 2, "If there are no outgoing requests, indicate that there are no outgoing requests") 

Here are the changes to indicate there are no outgoing requests if the outgoingRequests array is empty:

<replace start=20 end=22>
      {outgoingRequests?.length === 0 && (
        <p className="text-gray-600">You have no outgoing match requests at this time.</p>
      )}
</replace>

This results in the following updated component:

0 'use client'
1 import { useQuery } from 'convex/react'
2 import { api } from '@/convex/_generated/api'
3 import { Doc } from "@/convex/_generated/dataModel";
4 import useStoreUserEffect from '@/lib/useStoreUserEffect'
5 import MatchRequestCard from './MatchRequestCard'
6 
7 interface Props {
8   userId: string | null  
9 }
10
11 export default ({ userId }: Props) => {
12   const outgoingRequests = useQuery(api.backend.getOutgoingMatches, userId ? { } : 'skip')?.filter((request) => request.status === 'pending');
13
14   return (
15     <div className="mt-6">
16       <h2 className="text-xl font-semibold mb-4">Outgoing Match Requests</h2>
17       {outgoin

In [4]:
project.add_to_knowledge_base(1, 2)

{
  "pages": [
    {
      "title": "CreateProfile",
      "description": "Profile creation page to fill out name, background, LinkedIn profile, and technical cofounder status",
      "components": [
        {
          "name": "Footer",
          "description": "Footer with site links and copyright",
          "code": "import Link from 'next/link'\n\nexport default function Footer() {\n  return (\n    <footer className=\"bg-white\">\n      <div className=\"mx-auto max-w-7xl overflow-hidden py-20 px-6 sm:py-24 lg:px-8\">\n        <nav className=\"-mb-6 columns-2 sm:flex sm:justify-center sm:space-x-12\" aria-label=\"Footer\">\n          <div className=\"pb-6\">\n            <Link href=\"#\" className=\"text-sm leading-6 text-gray-600 hover:text-gray-900\">\n              About\n            </Link>\n          </div>\n          <div className=\"pb-6\">\n            <Link href=\"#\" className=\"text-sm leading-6 text-gray-600 hover:text-gray-900\">\n              Blog\n            </Link>